### Text Classification - Call summary

Given data of telephone conversation text and the category or purpose of the call, the problem is to classify the incoming calls based on the call data into one of those categories.

Multi-class classification problem

#### Importing Necessary Libraries

In [29]:
import os

import pandas as pd
import numpy as np

from sklearn import model_selection, preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.models import Sequential,Model
from keras.layers import Dense,Flatten,Embedding,Input,Conv1D,MaxPooling1D,LSTM,SpatialDropout1D,Dropout
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

from nltk.tokenize import word_tokenize


np.random.seed(1512)

In [30]:
PATH = os.getcwd()
os.listdir()

['.ipynb_checkpoints',
 'backup_submissions',
 'Batch39_Batch16_cute',
 'best_cnn_model.h5',
 'best_rnn_dropout_model_1.h5',
 'best_rnn_model.h5',
 'cnn_embed_model.csv',
 'Cute_CS7322C_Batch16.zip',
 'Cute_Embeddings_CNN.ipynb',
 'Cute_Embeddings_MLP.ipynb',
 'Cute_Embeddings_RNN.ipynb',
 'Cute_mlp_tfidf.ipynb',
 'description.pdf',
 'Final_ppt_Batch16.pptx',
 'glove.6B',
 'glove.6B.zip',
 'mlp_embed_model.csv',
 'mlp_embed_model_1.csv',
 'mlp_embed_model_2.csv',
 'NeuralNetworks_TextClassification.ipynb',
 'References',
 'rnn_do_model_1.csv',
 'rnn_do_model_2.csv',
 'rnn_do_model_3.csv',
 'rnn_model_1.csv',
 'rnn_model_2.csv',
 'samplesubmission.csv',
 'samplesubmissionbestmlp.csv',
 'samplesubmissiontest.csv',
 'SampleSubmission_mlp.csv',
 'test.csv',
 'Text Classification of Medical Industry.ipynb',
 'train.csv']

#### Reading the data

In [31]:
# Create pandas dataframe from csv input file
inputData = pd.read_csv("train.csv",index_col = "ID",na_values=" ")
inputData.columns

testData = pd.read_csv("test.csv",na_values=" ")
testData.columns

Index(['ID', 'converse'], dtype='object')

In [32]:
# replacing na values with string "blank calls"
inputData = inputData.replace(np.nan,"Blank calls")
testData = testData.replace(np.nan, "Blank Calls")

In [33]:
inputData['categories'].value_counts()

PRESCRIPTION     12077
APPOINTMENTS     11098
MISCELLANEOUS     9736
ASK_A_DOCTOR      9440
LAB               3457
JUNK                17
Name: categories, dtype: int64

#### Getting list of categories

In [34]:
categories = inputData['categories'].unique()
categories.sort()

In [35]:
# tokenizer 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(inputData['converse'])

word_Index = tokenizer.word_index
vocab_size = len(word_Index)+1

print("unique words ",vocab_size)

unique words  34770


### Train/Validation Split (85:15)

In [36]:
X_train, X_validate, y_train, y_validate = model_selection.train_test_split(inputData['converse'],inputData['categories'], random_state = 1512, stratify = inputData['categories'])

In [37]:
X_test = testData['converse']

In [38]:
#stop words removal
from nltk.corpus import stopwords
stopwordList = stopwords.words('english')

In [39]:
def removeStopWords(inputList):
   outputList = []
   for line in inputList:
        # tokenize line into words
        words = word_tokenize(line)
        # remove stop words
        selectedWords = [word for word in words if word not in stopwordList]
        # join back words into string (line)
        outputLine = ' '.join(selectedWords)
        # append back to the list to return
        outputList.append(outputLine)
   return outputList
        

In [40]:
X_train = removeStopWords(X_train)
X_validate = removeStopWords(X_validate)
X_test = removeStopWords(X_test)

In [41]:
X_train[0:5]

['patients running late mom appointments patient advised come office md approve mom stating mins late rna follow appointment moved phone completed appointment scheduled timephrase gina stewart rn',
 'fyi mom patients mom several times left several messages regards appts scheduled iv drugname doctor come one appointment gas come appts also request speak nurse regarding medications need advised mom adult tonya foreman lpn patients number someone answered phone lot hollering loud talking background asked jasmine kept saying hollering continued repeatedly said one said anything hung continue try patients either phone disconnected get junk like ask nicole let know drugname go speak personally need good phone number mother must told patient patient must make phone calls phone completed denny cook rn',
 'triage migraine request dheinj timephrase work phone patient triage adult migraines high pain level patients states bad migraine timephrase requesting inj dhe timephrase please advise timephr

In [42]:
# performing padding to make all sequences of similar length 
MAX_SEQUENCE_LENGTH = 150

train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen = MAX_SEQUENCE_LENGTH)
valid_seq = pad_sequences(tokenizer.texts_to_sequences(X_validate), maxlen = MAX_SEQUENCE_LENGTH)
test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen= MAX_SEQUENCE_LENGTH)

print(train_seq.shape)
print(valid_seq.shape)

(34368, 150)
(11457, 150)


#### Embeddings - Glove 6B 100d 

In [43]:
# embeddings - glove 6B 100d
Embeddings_Index = {}
for line in open('glove.6B/glove.6B.100d.txt', encoding= 'utf-8'):
    values = line.split()
    Embeddings_Index[values[0]] = np.asarray(values[1:],dtype='float32')



In [44]:
# map words to embeddings
embeddings_matrix = np.zeros((vocab_size,100))
for word,i in word_Index.items():
    vec = Embeddings_Index.get(word)
    if vec is not None:
        embeddings_matrix[i] = vec
        

In [45]:
embeddings_matrix.shape


(34770, 100)

In [46]:
# Creating Embedding layer
embeddingLayer = Embedding(vocab_size,
                           100,
                           weights = [embeddings_matrix],
                           input_length = MAX_SEQUENCE_LENGTH,
                           trainable = False)


In [47]:
# Function to convert probabilities to categories 
def convertClassToName(test_preds):
    y_pred = []
    for i in test_preds:
        num = np.argmax(i)
        y_pred.append(categories[num])
    return y_pred

# Function to write output of model to csv file
def writeOuputToCsv (model, filename):
    test_preds = model.predict(test_seq)
    y_pred = convertClassToName(test_preds)
    data = {'ID' : testData['ID'], 'categories' : y_pred}
    outputDf = pd.DataFrame(data= data, columns =['ID','categories'])
    outputDf.to_csv(filename,index=False)

### MLP model (1  hidden layer)

In [48]:
inputLayer = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
embedded_sequences = embeddingLayer(inputLayer)
dense_1 = Dense(64, activation='relu')(embedded_sequences)
flatten = Flatten()(dense_1)
preds = Dense(len(inputData['categories'].unique()), activation='softmax')(flatten)

mlp_embed_model = Model(inputLayer,preds)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [49]:
mlp_embed_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 100)          3477000   
_________________________________________________________________
dense_1 (Dense)              (None, 150, 64)           6464      
_________________________________________________________________
flatten_1 (Flatten)          (None, 9600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 57606     
Total params: 3,541,070
Trainable params: 64,070
Non-trainable params: 3,477,000
_________________________________________________________________


In [50]:
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_validate = encoder.fit_transform(y_validate)

In [51]:
y_train = to_categorical(y_train)
y_validate = to_categorical(y_validate)

In [52]:
mlp_embed_model.compile(optimizer="adam", loss="categorical_crossentropy",metrics =['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [53]:
hist_mlp_embed_model = mlp_embed_model.fit(train_seq, y_train, epochs=50, validation_data=(valid_seq,y_validate))

Train on 34368 samples, validate on 11457 samples
Epoch 1/50
34368/34368 [==============================] - 15s 438us/step - loss: 0.8546 - acc: 0.6857 - val_loss: 0.7229 - val_acc: 0.7452
Epoch 2/50
34368/34368 [==============================] - 16s 453us/step - loss: 0.6256 - acc: 0.7743 - val_loss: 0.7339 - val_acc: 0.7421
Epoch 3/50
34368/34368 [==============================] - 16s 479us/step - loss: 0.5370 - acc: 0.8044 - val_loss: 0.7494 - val_acc: 0.7437
Epoch 4/50
34368/34368 [==============================] - 17s 486us/step - loss: 0.4721 - acc: 0.8272 - val_loss: 0.8136 - val_acc: 0.7399
Epoch 5/50
34368/34368 [==============================] - 16s 480us/step - loss: 0.4190 - acc: 0.8472 - val_loss: 0.8347 - val_acc: 0.7399
Epoch 6/50
34368/34368 [==============================] - 16s 473us/step - loss: 0.3790 - acc: 0.8599 - val_loss: 0.8800 - val_acc: 0.7371
Epoch 7/50
34368/34368 [==============================] - 17s 485us/step - loss: 0.3443 - acc: 0.8752 - val_loss: 0.

In [54]:
writeOuputToCsv(mlp_embed_model,"mlp_embed_model_2.csv")

### CNN Model - Embeddings

In [55]:
## cnn model with embedding layer
input_seq = Input(shape = (MAX_SEQUENCE_LENGTH,), dtype = "int32")
embedding_seq = embeddingLayer(input_seq)
x = Conv1D(64, 5, activation='relu')(embedding_seq)
x = MaxPooling1D(4)(x)
x = Conv1D(64, 5, activation='relu')(x)
x = MaxPooling1D(4)(x)
x = Conv1D(64, 5, activation='relu')(x)
x = MaxPooling1D(4)(x)  # global max pooling
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
preds = Dense(len(categories), activation='softmax')(x)

cnn_embed_model = Model(input_seq,preds)


In [56]:
cnn_embed_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [57]:
cnn_embed_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 100)          3477000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 146, 64)           32064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 36, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 32, 64)            20544     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 8, 64)             0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 4, 64)             20544     
__________

In [58]:
callbacks = [EarlyStopping(monitor='val_acc', patience=3),
             ModelCheckpoint(filepath='best_cnn_model.h5', monitor='val_acc', save_best_only=True)]
hist_cnn_embed_model = cnn_embed_model.fit(train_seq,y_train,epochs = 25,callbacks=callbacks,validation_data=(valid_seq,y_validate))

Train on 34368 samples, validate on 11457 samples
Epoch 1/25
34368/34368 [==============================] - 54s 2ms/step - loss: 0.7604 - acc: 0.7220 - val_loss: 0.5925 - val_acc: 0.7844
Epoch 2/25
34368/34368 [==============================] - 49s 1ms/step - loss: 0.5609 - acc: 0.7968 - val_loss: 0.5662 - val_acc: 0.7921
Epoch 3/25
34368/34368 [==============================] - 51s 1ms/step - loss: 0.4949 - acc: 0.8166 - val_loss: 0.5684 - val_acc: 0.7912
Epoch 4/25
34368/34368 [==============================] - 57s 2ms/step - loss: 0.4491 - acc: 0.8355 - val_loss: 0.5817 - val_acc: 0.7880
Epoch 5/25
34368/34368 [==============================] - 65s 2ms/step - loss: 0.4005 - acc: 0.8534 - val_loss: 0.5872 - val_acc: 0.7877


In [59]:
best_cnn_model = load_model('best_cnn_model.h5')
writeOuputToCsv(best_cnn_model,"cnn_embed_model.csv")

### RNN- LSTM Embeddings - Model1

In [60]:
#simple rnn with embedding layer
rnn_model = Sequential()
rnn_model.add(Embedding(vocab_size,
                           100,
                           weights = [embeddings_matrix],
                           input_length = MAX_SEQUENCE_LENGTH,
                           trainable = False))
rnn_model.add(LSTM(100))
rnn_model.add(Dense(len(categories),activation='softmax'))

In [61]:
rnn_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [62]:
rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 100)          3477000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 606       
Total params: 3,558,006
Trainable params: 81,006
Non-trainable params: 3,477,000
_________________________________________________________________


In [63]:
callbacks = [EarlyStopping(monitor='val_acc', patience=3),
             ModelCheckpoint(filepath='best_rnn_model.h5', monitor='val_acc', save_best_only=True)]
hist_rnn_model = rnn_model.fit(train_seq,y_train,epochs = 50,callbacks= callbacks, validation_data=(valid_seq,y_validate))

Train on 34368 samples, validate on 11457 samples
Epoch 1/50
34368/34368 [==============================] - 165s 5ms/step - loss: 0.8879 - acc: 0.6720 - val_loss: 0.6653 - val_acc: 0.7586
Epoch 2/50
34368/34368 [==============================] - 176s 5ms/step - loss: 0.6189 - acc: 0.7737 - val_loss: 0.5812 - val_acc: 0.7856
Epoch 3/50
34368/34368 [==============================] - 197s 6ms/step - loss: 0.5512 - acc: 0.7960 - val_loss: 0.5596 - val_acc: 0.7950
Epoch 4/50
34368/34368 [==============================] - 197s 6ms/step - loss: 0.5125 - acc: 0.8070 - val_loss: 0.5324 - val_acc: 0.7999
Epoch 5/50
34368/34368 [==============================] - 219s 6ms/step - loss: 0.4851 - acc: 0.8164 - val_loss: 0.5360 - val_acc: 0.7997
Epoch 6/50
34368/34368 [==============================] - 179s 5ms/step - loss: 0.4629 - acc: 0.8234 - val_loss: 0.5187 - val_acc: 0.8074
Epoch 7/50
34368/34368 [==============================] - 186s 5ms/step - loss: 0.4412 - acc: 0.8311 - val_loss: 0.5318 - 

In [64]:
best_rnn_model = load_model('best_rnn_model.h5')
writeOuputToCsv(best_rnn_model,"rnn_model_2.csv")

### RNN- LSTM Embeddings - Model2 ( More dense layers)


In [65]:
#rnn including dropout
input_seq = Input(shape = (MAX_SEQUENCE_LENGTH,), dtype = "int32")
embedding_seq = embeddingLayer(input_seq)
embedding_seq = SpatialDropout1D(0.3)(embedding_seq)
#LSTM layer
lstm_layer = LSTM(100)(embedding_seq)
#dense layers
output_layer1 = Dense(50, activation="relu")(lstm_layer)
output_layer1 = Dropout(0.25)(output_layer1)
preds = Dense(len(categories), activation='softmax')(output_layer1)

rnn_dropout_model = Model(input_seq,preds)
rnn_dropout_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [66]:
rnn_dropout_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 100)          3477000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 150, 100)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_6 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 306       
Total para

In [67]:
callbacks = [EarlyStopping(monitor='val_acc', patience=2),
             ModelCheckpoint(filepath='best_rnn_dropout_model_1.h5', monitor='val_acc', save_best_only=True)]
hist_rnn_dropout_model = rnn_dropout_model.fit(train_seq,y_train,epochs = 50,callbacks= callbacks, validation_data=(valid_seq,y_validate))

Train on 34368 samples, validate on 11457 samples
Epoch 1/50
34368/34368 [==============================] - 162s 5ms/step - loss: 0.9526 - acc: 0.6458 - val_loss: 0.6742 - val_acc: 0.7593
Epoch 2/50
34368/34368 [==============================] - 159s 5ms/step - loss: 0.6799 - acc: 0.7574 - val_loss: 0.5870 - val_acc: 0.7814
Epoch 3/50
34368/34368 [==============================] - 157s 5ms/step - loss: 0.6134 - acc: 0.7779 - val_loss: 0.5545 - val_acc: 0.7894
Epoch 4/50
34368/34368 [==============================] - 159s 5ms/step - loss: 0.5756 - acc: 0.7896 - val_loss: 0.5361 - val_acc: 0.7976
Epoch 5/50
34368/34368 [==============================] - 163s 5ms/step - loss: 0.5532 - acc: 0.7951 - val_loss: 0.5356 - val_acc: 0.7990
Epoch 6/50
34368/34368 [==============================] - 162s 5ms/step - loss: 0.5390 - acc: 0.7991 - val_loss: 0.5110 - val_acc: 0.8087
Epoch 7/50
34368/34368 [==============================] - 163s 5ms/step - loss: 0.5189 - acc: 0.8077 - val_loss: 0.5117 - 

In [68]:
best_rnn_do_model = load_model('best_rnn_dropout_model_1.h5')
writeOuputToCsv(best_rnn_do_model,"rnn_do_model_2.csv")